In [1]:
from datasets import Dataset 
from ragas.metrics import faithfulness
from ragas import evaluate
import os
from datasets import Dataset
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatZhipuAI
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
import os
os.environ["ZHIPUAI_API_KEY"] = "661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs"
chat = ChatZhipuAI(
    model="glm-4-0520",
    temperature=0.5,
)


In [17]:
from langchain_community.chat_models import ChatTongyi

tongyi_chat = ChatTongyi(
    model="qwen-max-0428",
    # top_p="...",
    api_key="",
    # other params...
)

In [18]:
messages = [
    ("system", "你是一名专业的翻译家，可以将用户的中文翻译为英文。"),
    ("human", "我喜欢编程。"),
]
tongyi_chat.invoke(messages)

AIMessage(content='I enjoy programming.', response_metadata={'model_name': 'qwen-max-0428', 'finish_reason': 'stop', 'request_id': 'd60f23ba-5360-9039-a312-6e41c7399e12', 'token_usage': {'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}}, id='run-8db69089-4e92-45af-90b9-d48480f297f8-0')

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

# ✅ Pydantic class
class multiply(BaseModel):
    """Return product of 'x' and 'y'."""
    x: float = Field(..., description="First factor")
    y: float = Field(..., description="Second factor")
    
# ✅ LangChain tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y
    
# ✅ Function

def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y-x
    
# ✅ OpenAI-format dict
# Could also pass in a JSON schema with "title" and "description" 
add = {
  "name": "add",
  "description": "Add 'x' and 'y'.",
  "parameters": {
    "type": "object",
    "properties": {
      "x": {"type": "number", "description": "First number to add"},
      "y": {"type": "number", "description": "Second number to add"}
    },
    "required": ["x", "y"]
  }
}

In [23]:
llm_with_tools = tongyi_chat.bind_tools([exponentiate, add, subtract])

In [ ]:
llm_with_tools.kwargs

In [27]:
query = 'what is 3 ** 12?'
tr = llm_with_tools.invoke(query).tool_calls

In [28]:
for tc in tr:
    print(tc)

{'name': 'exponentiate', 'args': {'x': 3, 'y': 12}, 'id': ''}


In [30]:
selected_tool = {"add":add, "multiply":multiply, "exponentiate": exponentiate}[tc['name'].lower()]

In [33]:
selected_tool.invoke(tc["args"])

531441.0

In [6]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4-0520",
    openai_api_key="661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

In [9]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [10]:
multiply.invoke({"x": 4, "y": 5})

20.0

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
rendered_tools

'multiply(x: float, y: float) -> float - Multiply two numbers together.\nadd(x: int, y: int) -> int - Add two numbers.'

In [12]:
system_prompt =  f"""\
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
 [("system", system_prompt),  ("user",  "{input}")])

In [13]:
system_prompt

"You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nmultiply(x: float, y: float) -> float - Multiply two numbers together.\nadd(x: int, y: int) -> int - Add two numbers.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.\n\nThe `arguments` should be a dictionary, with keys corresponding to the argument names and the values corresponding to the requested values.\n"

In [15]:
chain = prompt | llm
message = chain.invoke({"input": "4的平方是多少？"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

To calculate the square of 4, you would multiply 4 by 4. The tool to use for this operation is `multiply`. Here is the response in the requested format:

```json
{
  "name": "multiply",
  "arguments": {
    "x": 4.0,
    "y": 4.0
  }
}
```


In [16]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "4的平方是多少？"})

{'name': 'multiply', 'arguments': {'x': 4.0, 'y': 4.0}}

In [17]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

In [18]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

In [19]:
chain = prompt | llm | JsonOutputParser() | invoke_tool
chain.invoke({"input": "6+89=？"})

95

In [20]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = []
dc_name = ["soybean_konw.pdf", "soybean2.pdf"]
for tmp_name in dc_name:
    # print(len(PyPDFLoader(tmp_name).load()))
    documents += PyPDFLoader(tmp_name).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents[:])
for idx, text in enumerate(texts):
    text.metadata["id"] = idx
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

top_k = 10

model_name = '/mnt/workspace/.cache/modelscope/hub/maple77/zpoint_large_embedding_zh'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = Chroma(persist_directory="soybean_db2", embedding_function=hf)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": top_k}
)

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "soybean_info_search",
    "search for information about soybean"
)

In [22]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add, retriever_tool]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
soybean_info_search
search for information about soybean
{'query': {'title': 'Query', 'description': 'query to look up in retriever', 'type': 'string'}}


In [23]:
llm.bind_tools(tools)

NotImplementedError: 